In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
class RandomForestModel:
    def __init__(self, train_df, test_df, targets):
        """
        Initialize the RandomForestModel.

        Parameters:
        - data: The dataset for modeling.
        - targets: The target variables to predict.
        """
        self.train_df = train_df
        self.test_df = test_df
        self.targets = targets
        self.model = None
        self.best_params = None
        self.best_score = None
        self.error_metrics = None
        self.pred = None
        self.importances = None
        self.trees = []
        self.graphs = []

    def parameters(self):
        """
        Get all the instance variables in a dictionary format.

        Returns:
        - params_dict: A dictionary with instance variable names as keys and their values as values.
        """
        params_dict = {
            "targets": self.targets,
            "best_params": self.best_params,
            "best_score": self.best_score,
            "error_metrics": self.error_metrics,
            "importances": self.importances,
        }
        return params_dict

    def update_targets(self, targets):
        """
        Update the target variables for modeling.

        Parameters:
        - targets: The new target variables to set.
        """
        self.targets = targets

    def clean_data(self):

        for df in [self.train_df, self.test_df]:

            df['date'] = pd.to_datetime(df['date'])
            df['hour'] = df['date'].dt.hour
            df['day_of_week'] = df['date'].dt.dayofweek
            df['day_of_year'] = df['date'].dt.dayofyear
            df['month'] = df['date'].dt.month
            df.drop('date', axis=1, inplace=True)

        return self.train_df, self.test_df

    def prepare_data(self, exempt=[]):
        """
        Prepare the datasets for modeling.

        Parameters:
        - targets: The target variables to prepare the data for.

        Returns:
        - train_df, test_df: The prepared training and testing datasets.
        """
        for target in self.targets:

            for df in [self.train_df, self.test_df]:

                # Shift by Date Cycles
                df[f'{target[:2]}_next_hour'] = df[target].shift(-1)
                df[f'{target[:2]}_next_day'] = df[target].rolling(window=24).sum()
                df[f'{target[:2]}_next_weekday'] = df[target].rolling(window=7 * 24).sum()
                df[f'{target[:2]}_next_month'] = df[target].rolling(window=30 * 24).sum()

                # Lag by Date Cycles
                df[f'{target[:2]}_inverse_hour'] = df[target].shift(1)
                df[f'{target[:2]}_inverse_next_day'] = df[target].diff(24)
                df[f'{target[:2]}_inverse_next_weekday'] = df[target].diff(7 * 24)
                df[f'{target[:2]}_inverse_next_month'] = df[target].diff(30 * 24)

                df.dropna(inplace=True)

                # Rolling Mean by Date Cycles
                df[f"{target[:2]}_6hour_mean"] = df[target].rolling(6).mean()
                df[f"{target[:2]}_12hour_mean"] = df[target].rolling(12).mean()
                df[f"{target[:2]}_24hour_mean"] = df[target].rolling(24).mean()
                df[f"{target[:2]}_week_mean"] = df[target].rolling(24*7).mean()
                df[f"{target[:2]}_30day_mean"] = df[target].rolling(24*30).mean()

                # Rolling Min by Date Cycles
                df[f"{target[:2]}_6hour_min"] = df[target].rolling(6).min()
                df[f"{target[:2]}_12hour_min"] = df[target].rolling(12).min()
                df[f"{target[:2]}_24hour_min"] = df[target].rolling(24).min()
                df[f"{target[:2]}_week_min"] = df[target].rolling(24*7).min()
                df[f"{target[:2]}_30day_min"] = df[target].rolling(24*30).min()

                # Rolling Max by Date Cycles
                df[f"{target[:2]}_6hour_max"] = df[target].rolling(6).max()
                df[f"{target[:2]}_12hour_max"] = df[target].rolling(12).max()
                df[f"{target[:2]}_24hour_max"] = df[target].rolling(24).max()
                df[f"{target[:2]}_week_max"] = df[target].rolling(24*7).max()
                df[f"{target[:2]}_30day_max"] = df[target].rolling(24*30).max()

                # Rolling Standard Deviation by Date Cycles
                df[f"{target[:2]}_6hour_std"] = df[target].rolling(6).std()
                df[f"{target[:2]}_12hour_std"] = df[target].rolling(12).std()
                df[f"{target[:2]}_24hour_std"] = df[target].rolling(24).std()
                df[f"{target[:2]}_week_std"] = df[target].rolling(24*7).std()
                df[f"{target[:2]}_30day_std"] = df[target].rolling(24*30).std()

        self.features = [feature for feature in self.train_df.columns if feature not in self.targets or exempt]

        return self.train_df, self.test_df, self.features
    
    def calculate_wae_rmse(self, reference, predictions):
        """
        Calculate the Weighted Absolute Error (WAE), Root Mean Square Error (RMSE),
        and their respective accuracies for a single target.

        Parameters:
        - reference: The reference dataset containing true values.
        - predictions: The predicted values to compare to.

        Returns:
        - error_metrics: Dictionary containing WAE, RMSE, and their accuracies for the specified target.
        """
        error_metrics = {}

        # Convert DataFrame or Series to NumPy arrays
        y_true = reference.values.astype(float)  # Convert to float if not already
        y_pred = predictions.astype(float)

        # Calculate Weighted Absolute Error (WAE)
        wae = np.sum(np.abs(y_true - y_pred))
        error_metrics['wae'] = wae

        # Calculate Root Mean Square Error (RMSE)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        error_metrics['rmse'] = rmse

        # Calculate the accuracy for WAE (WAE Accuracy)
        wae_accuracy = 100 - (wae / np.sum(np.abs(y_true)) * 100)
        error_metrics['wae_accuracy'] = wae_accuracy

        # Calculate the accuracy for RMSE (RMSE Accuracy)
        rmse_accuracy = 100 - (rmse / np.std(y_true) * 100)
        error_metrics['rmse_accuracy'] = rmse_accuracy

        self.error_metrics = error_metrics

        return error_metrics

    def tune_random_forest_hyperparameters(self, X, y):
        """
        Tune the hyperparameters of a Random Forest model using GridSearchCV.

        Parameters:
        - X: Input features.
        - y: Target variable.

        Returns:
        - best_params: The best hyperparameters found by GridSearchCV.
        - best_score: The best score achieved with the best hyperparameters.
        """
        param_grid = {
            'n_estimators': [50, 100, 150],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt', 'log2']
        }

        rf = RandomForestRegressor(random_state=0, n_jobs=6)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
        grid_search.fit(X, y)

        self.best_params = grid_search.best_params_
        self.best_score = grid_search.best_score_

        return self.best_params, self.best_score

    def fit_model(self):
        """
        Fit a RandomForestRegressor model to the training dataset.

        Returns:
        - output: A dictionary containing model-related information.
        """
        train_df = self.train_df.copy()

        imputer = SimpleImputer()
        self.Xtr = imputer.fit_transform(train_df[self.features])
        self.ytr = train_df[self.targets]

        best_params, best_score = self.tune_random_forest_hyperparameters(self.Xtr, self.ytr)

        mdl = RandomForestRegressor(n_estimators=best_params.get('n_estimators', 100),
                                    max_depth=best_params.get('max_depth', None),
                                    min_samples_split=best_params.get('min_samples_split', 2),
                                    min_samples_leaf=best_params.get('min_samples_leaf', 1),
                                    max_features=best_params.get('max_features', 'auto'),
                                    random_state=best_params.get('random_state', 0),
                                    n_jobs=best_params.get('n_jobs', 6))

        mdl.fit(self.Xtr, self.ytr)

        self.model = mdl
        self.params = best_params
        self.score = best_score

        return self.parameters()

    def predict(self):
        """
        Predict target values for the testing dataset and compare results.

        Returns:
        - pred: Predicted values for each target column.
        """
        test_df = self.test_df.copy()

        # Fit the imputer on your training data first
        imputer = SimpleImputer()
        imputer.fit(self.train_df[self.features])

        # Transform the test data using the fitted imputer
        Xtest = imputer.transform(test_df[self.features])

        # Make predictions
        pred = self.model.predict(Xtest)

        self.pred = pred
        return self.pred

    def print_performance(self, target, sig_level=0.05):

        important_features = [feature for feature in self.features if self.importances.get(feature, 0) > sig_level]
        df = self.train_df.copy().dropna(subset=self.features).fillna(0)
        important_df = df[important_features]
        target_df = df[target]

        # Train the random forest
        rf_most_important = RandomForestRegressor(n_estimators=1000, random_state=42)
        rf_most_important.fit(important_df, target_df)

        # Prepare the test data using the same important features
        test_important_df = df[important_features].fillna(0)

        # Make predictions
        predictions = rf_most_important.predict(test_important_df)

        # Calculate WAE, RMSE, and their respective accuracies
        error_metrics = self.calculate_wae_rmse(target_df, predictions)

        # Display the performance metrics
        print('Mean Absolute Error (WAE):', round(error_metrics['wae'], 4))
        print('WAE Accuracy:', round(error_metrics['wae_accuracy'], 4), '%.')

        print('Root Mean Square Error (RMSE):', round(error_metrics['rmse'], 4))
        print('RMSE Accuracy:', round(error_metrics['rmse_accuracy'], 4), '%.')

    def visualize_tree(self, tree_index, dot_loc, png_loc=None, new_params=None):
        """
        Visualize a decision tree from the random forest model.

        Args:
            tree_index (int): Index of the tree to visualize.
            dot_loc (str): File path to save the DOT file.
            png_loc (str, optional): File path to save the visualization as a PNG file.

        Returns:
        - graph: A Pydot graph representing the decision tree.
        """
        if new_params is not None:

            tree_model = RandomForestRegressor(
                n_estimators=new_params.get('n_estimators', self.params.get('n_estimators', 100)),
                max_depth=new_params.get('max_depth', self.params.get('max_depth', None)),
                min_samples_split=new_params.get('min_samples_split', self.params.get('min_samples_split', 2)),
                min_samples_leaf=new_params.get('min_samples_leaf', self.params.get('min_samples_leaf', 1)),
                max_features=new_params.get('max_features', self.params.get('max_features', 'auto')),
                random_state=new_params.get('random_state', self.params.get('random_state', 0)),
                n_jobs=new_params.get('n_jobs', self.params.get('n_jobs', 6))
            )

            tree_model.fit(self.Xtr, self.ytr)

        else:
            tree_model = self.model

        if tree_index < 0 or tree_index >= len(tree_model.estimators_):
            raise ValueError(f"Invalid tree_index. It should be in the range [0, {len(tree_model.estimators_)-1}].")
        
        tree = tree_model.estimators_[tree_index]

        export_graphviz(
            tree,
            out_file=dot_loc,
            feature_names=self.features,
            rounded=True,
            precision=1
        )

        (tree_chart, ) = pydot.graph_from_dot_file(dot_loc)

        if png_loc is not None:
            tree_chart.write_png(png_loc)

        self.trees.append(tree_chart)

        return tree_chart

    def print_importances(self, decimal_places=4):

        importances = {}
        
        feature_importances = [(feature, round(importance, decimal_places)) for feature, importance in zip(self.features, self.model.feature_importances_)]
        feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

        for feature, importance in feature_importances:
            importances[feature] = importance
            print(f'Variable: {feature:20} Importance: {importance:.{decimal_places}f}')

        self.importances = importances

        
    def graph_importances(self, sig_val, loc):
        plt.style.use('ggplot')

        features = [feature for feature, importance in self.importances.items() if importance > sig_val]
        importances = [importance for feature, importance in self.importances.items() if importance > sig_val]

        # List of x locations for plotting
        x_values = list(range(len(importances)))

        # Make a bar chart
        plt.bar(x_values, importances, orientation='vertical')

        # Tick labels for x axis
        plt.xticks(x_values, features, rotation=90)

        # Axis labels and title
        plt.ylabel('Importance')
        plt.xlabel('Variable')
        plt.title('Variable Importances')

        plt.savefig(loc)

        self.graphs.append(plt)

        return plt

In [12]:
from collections import Counter

class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

class DecisionTree:
    '''
    Class which implements a decision tree classifier algorithm.
    '''
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(s):
        '''
        Helper function, calculates entropy from an array of integer values.
        
        :param s: list
        :return: float, entropy value
        '''
        # Convert to integers to avoid runtime errors
        counts = np.bincount(np.array(s, dtype=np.int64))

        # Probabilities of each class label
        percentages = counts / len(s)

        # Caclulate entropy
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    def _information_gain(self, parent, left_child, right_child):
        '''
        Helper function, calculates information gain from a parent and two child nodes.
        
        :param parent: list, the parent node
        :param left_child: list, left child of a parent
        :param right_child: list, right child of a parent
        :return: float, information gain
        '''
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        
        # One-liner which implements the previously discussed formula
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    
    def _best_split(self, X, y):
        '''
        Helper function, calculates the best split for given features and target
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # For every dataset feature
        for f_idx in range(n_cols):

            X_curr = X[:, f_idx]
            # For every unique value of that feature
            for threshold in np.unique(X_curr):

                # Construct a dataset and split it to the left and right parts
                # Left part includes records lower or equal to the threshold
                # Right part includes records higher than the threshold
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                # Do the calculation only if there's data in both subsets
                if len(df_left) > 0 and len(df_right) > 0:

                    # Obtain the value of the target variable for subsets
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    # Caclulate the information gain and save the split parameters
                    # if the current split if better then the previous best
                    gain = self._information_gain(y, y_left, y_right)

                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain

        return best_split
    
    def _build(self, X, y, depth=0):
        '''
        Helper recursive function, used to build a decision tree from the input data.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :param depth: current depth of a tree, used as a stopping criteria
        :return: Node
        '''
        n_rows, n_cols = X.shape
        
        # Check to see if a node should be leaf node
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            # Get the best split
            best = self._best_split(X, y)
            # If the split isn't pure
            if best['gain'] > 0:
                # Build a tree on the left
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right,
                    gain=best['gain']
                )
            
        # Leaf node - value is the most common target value 
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        '''
        Function used to train a decision tree classifier model.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: None
        '''
        # Call a recursive function to build the tree
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):
        '''
        Helper recursive function, used to predict a single instance (tree traversal).
        
        :param x: single observation
        :param tree: built tree
        :return: float, predicted class
        '''
        # Leaf node
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # Go to the left
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # Go to the right
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        Function used to classify new instances.
        
        :param X: np.array, features
        :return: np.array, predicted classes
        '''
        # Call the _predict() function for every observation
        return [self._predict(x, self.root) for x in X]
    
class RandomForest:
    '''
    A class that implements Random Forest algorithm from scratch.
    '''
    def __init__(self, num_trees=25, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        # Will store individually trained decision trees
        self.decision_trees = []
        
    @staticmethod
    def _sample(X, y):
        '''
        Helper function used for boostrap sampling.
        
        :param X: np.array, features
        :param y: np.array, target
        :return: tuple (sample of features, sample of target)
        '''
        n_rows, n_cols = X.shape
        # Sample with replacement
        samples = np.random.choice(a=n_rows, size=n_rows, replace=True)
        return X[samples], y[samples]
        
    def fit(self, X, y):
        '''
        Trains a Random Forest classifier.
        
        :param X: np.array, features
        :param y: np.array, target
        :return: None
        '''
        # Reset
        if len(self.decision_trees) > 0:
            self.decision_trees = []
            
        # Build each tree of the forest
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = DecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                # Obtain data sample
                _X, _y = self._sample(X, y)
                # Train
                clf.fit(_X, _y)
                # Save the classifier
                self.decision_trees.append(clf)
                num_built += 1
            except Exception as e:
                continue
    
    def predict(self, X):
        '''
        Predicts class labels for new data instances.
        
        :param X: np.array, new instances to predict
        :return: 
        '''
        # Make predictions with every tree in the forest
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
        
        # Reshape so we can find the most common value
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        
        # Use majority voting for the final prediction
        predictions = []
        for preds in y:
            counter = Counter(x)
            predictions.append(counter.most_common(1)[0][0])
        return predictions

In [13]:
test_sample = pd.read_csv('../datasets/representative_samples/test_non-violent_max.csv')
train_sample = pd.read_csv('../datasets/representative_samples/train_non-violent_max.csv')

In [14]:
model = RandomForestModel(train_sample, test_sample, ['non-violent'])

In [15]:
model.clean_data()

(       non-violent  violent  train_rides  bike_rides  lighting  \
 0              0.0      1.0      39467.0        14.0      22.0   
 1              2.0      1.0      39467.0         2.0      22.0   
 2              0.0      2.0      39467.0         0.0      22.0   
 3              1.0      1.0      39467.0         0.0      22.0   
 4              1.0      0.0      39467.0         1.0      22.0   
 ...            ...      ...          ...         ...       ...   
 25646          0.0      0.0      68996.0        12.0       0.0   
 25647          3.0      0.0      68996.0         9.0       0.0   
 25648          1.0      0.0      68996.0         5.0       0.0   
 25649          0.0      0.0      68996.0         8.0       0.0   
 25650          1.0      0.0      68996.0        17.0       0.0   
 
        vacant_buildings  hour  day_of_week  day_of_year  month  
 0                  31.0     1            4            1      1  
 1                  31.0     2            4            1      

In [16]:
model.prepare_data()

(       non-violent  violent  train_rides  bike_rides  lighting  \
 720            0.0      0.0      35887.0        15.0      20.0   
 721            2.0      1.0     145929.0         1.0      20.0   
 722            0.0      1.0     145929.0         3.0      20.0   
 723            0.0      0.0     145929.0         2.0      20.0   
 724            0.0      0.0     145929.0         5.0      20.0   
 ...            ...      ...          ...         ...       ...   
 25645          2.0      0.0      68996.0        13.0       0.0   
 25646          0.0      0.0      68996.0        12.0       0.0   
 25647          3.0      0.0      68996.0         9.0       0.0   
 25648          1.0      0.0      68996.0         5.0       0.0   
 25649          0.0      0.0      68996.0         8.0       0.0   
 
        vacant_buildings  hour  day_of_week  day_of_year  month  ...  \
 720                31.0    22            6           31      1  ...   
 721                31.0     0            0       

In [17]:
def test_accuracy(columns):
  for i in columns:
    globals()['test_' + i + '_avg'] = pd.read_csv('../datasets/representative_samples/test_' + i + '_avg.csv')
    globals()['test_' + i + '_max'] = pd.read_csv('../datasets/representative_samples/test_' + i + '_max.csv')
    globals()['test_' + i + '_min'] = pd.read_csv('../datasets/representative_samples/test_' + i + '_min.csv')
    
    globals()['train_' + i + '_avg'] = pd.read_csv('../datasets/representative_samples/train_' + i + '_avg.csv')
    globals()['train_' + i + '_max'] = pd.read_csv('../datasets/representative_samples/train_' + i + '_max.csv')
    globals()['train_' + i + '_min'] = pd.read_csv('../datasets/representative_samples/train_' + i + '_min.csv')

    globals()[i + '_avg_model'] = RandomForestModel(globals()['train_' + i + '_avg'], globals()['test_' + i + '_avg'], [i])
    globals()[i + '_max_model'] = RandomForestModel(globals()['train_' + i + '_max'], globals()['test_' + i + '_max'], [i])
    globals()[i + '_min_model'] = RandomForestModel(globals()['train_' + i + '_min'], globals()['test_' + i + '_min'], [i])

    globals()[i + '_avg_model'].clean_data()
    globals()[i + '_max_model'].clean_data()
    globals()[i + '_min_model'].clean_data()

    globals()[i + '_avg_model'].prepare_data()
    globals()[i + '_max_model'].prepare_data()
    globals()[i + '_min_model'].prepare_data()

    globals()[i + '_avg_model'].fit_model()
    globals()[i + '_max_model'].fit_model()
    globals()[i + '_min_model'].fit_model()

In [18]:
test_accuracy(['non-violent', 'violent'])

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, 

/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 

[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.5s


/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.8s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.9s


...

/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   1.0s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=150; total time=   1.6s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   1.1s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.9s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=150; total time=   1.5s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=150; total time=   1.5s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   1.0s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.9s


/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   1.0s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   1.0s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.9s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.9s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.4s


/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
326 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 638, in _validate_params
    validat

In [19]:
def make_predictions(columns):
  for i in columns:
    globals()[i + '_avg_model'].predict()
    globals()[i + '_max_model'].predict()
    globals()[i + '_min_model'].predict()

    print(i + '_avg_model 0.05 accuracy')
    globals()[i + '_avg_model'].print_performance(i)
    print()
    print(i + '_avg_model 0.01 accuracy')
    globals()[i + '_avg_model'].print_performance(i, sig_level=0.01)
    print()
    print(i + '_avg_model all-cols accuracy')
    globals()[i + '_avg_model'].print_performance(i, sig_level=-1)
    print()
    print(i + '_max_model 0.05 accuracy')
    globals()[i + '_max_model'].print_performance(i)
    print()
    print(i + '_max_model 0.01 accuracy')
    globals()[i + '_max_model'].print_performance(i, sig_level=0.01)
    print()
    print(i + '_max_model all-cols accuracy')
    globals()[i + '_max_model'].print_performance(i, sig_level=-1)
    print()
    print(i + '_min_model 0.05 accuracy')
    globals()[i + '_min_model'].print_performance(i)
    print()
    print(i + '_min_model 0.01 accuracy')
    globals()[i + '_min_model'].print_performance(i, sig_level=0.01)
    print()
    print(i + '_min_model all-cols accuracy')
    globals()[i + '_min_model'].print_performance(i, sig_level=-1)
    print()

In [25]:
def generate_trees(columns):
    for i in columns:
        globals()[i + '_avg_model'].visualize_tree(3, f'graphs/trees/dots/{i}_avg_model.dot', png_loc=f'graphs/trees/pngs/{i}_avg_model.png', new_params=None)
        globals()[i + '_max_model'].visualize_tree(3, f'graphs/trees/dots/{i}_max_model.dot', png_loc=f'graphs/trees/pngs/{i}_max_model.png', new_params=None)
        globals()[i + '_min_model'].visualize_tree(3, f'graphs/trees/dots/{i}_min_model.dot', png_loc=f'graphs/trees/pngs/{i}_min_model.png', new_params=None)

        small_tree = {
            'n_estimators' : 5,
            'max_depth' : 3,
        }

        globals()[i + '_avg_model'].visualize_tree(3, f'graphs/trees/dots/{i}_avg_model_st.dot', png_loc=f'graphs/trees/small_trees/{i}_avg_model_st.png', new_params=small_tree)
        globals()[i + '_max_model'].visualize_tree(3, f'graphs/trees/dots/{i}_max_model_st.dot', png_loc=f'graphs/trees/small_trees/{i}_max_model_st.png', new_params=small_tree)
        globals()[i + '_min_model'].visualize_tree(3, f'graphs/trees/dots/{i}_min_model_st.dot', png_loc=f'graphs/trees/small_trees/{i}_min_model_st.png', new_params=small_tree)

In [26]:
generate_trees(['non-violent', 'violent'])

/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 

In [ ]:
make_predictions(['non-violent', 'violent'])

In [ ]:
# WAE = 97.65%
# RMSE = 94.86%